In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [3]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
--CÓDIGO QUE RETORNA A TABELA JÁ COM O NOME DOS MEDICAMENTOS (MAS QUE É MUITO DEMORADO):
--DROP VIEW IF EXISTS Drug_Couple;
--CREATE VIEW Drug_Couple AS
--SELECT DA.name AS drugA, DB.name AS drugB FROM Drug DA, Drug DB, DrugUse DUA, DrugUse DUB WHERE DA.code = DUA.codeDrug AND DB.code = DUB.codeDrug AND DUA.idPerson = DUB.idPerson AND DUA.codeDrug <> DUB.codeDrug;

--SELECT DC.drugA, DC.drugB, COUNT(*) NUM FROM Drug_Couple DC GROUP BY DC.drugA, DC.drugB 
--HAVING NUM > 30;

--CÓDIGO QUE RETORNA A TABELA COM OS CÓDIGOS DOS MEDICAMENTOS:

DROP VIEW IF EXISTS Drug_Couple;
CREATE VIEW Drug_Couple AS
SELECT DUA.codeDrug AS codeA, DUB.codeDrug AS codeB FROM DrugUse DUA, DrugUse DUB 
WHERE DUA.idPerson = DUB.idPerson AND DUA.codeDrug <> DUB.codeDrug;

SELECT codeA, codeB, COUNT(*) NUM FROM Drug_Couple GROUP BY codeA, codeB 
HAVING NUM > 30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [3]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [8]:
DROP VIEW IF EXISTS GrupoMais30;
CREATE VIEW GrupoMais30 AS
SELECT * FROM DrugInteraction DI WHERE DI.weight > 30;

--SELECT * FROM GrupoMais30;

SELECT GM.drugA, GM.drugB, CAST(GM.WEIGHT AS DOUBLE)/9073 Suporte FROM GrupoMais30 GM ORDER BY Suporte DESC;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [ ]:
--Muuuito provavelmente funciona, porém esbarra no limite de memoria do Binder
DROP VIEW IF EXISTS GrupoMais30_Ap;
DROP VIEW IF EXISTS GrupoMais30;
CREATE VIEW GrupoMais30 AS
SELECT * FROM DrugInteraction DI WHERE DI.weight > 30;

--SELECT GM.drugA, GM.drugB, CAST(GM.WEIGHT AS DOUBLE)/ APARICOES Confianca FROM GrupoMais30 GM
--WHERE APARICOES = (SELECT COUNT(*) FROM DrugInteraction DI GROUP BY GM.drugA);

CREATE VIEW GrupoMais30_Ap AS
SELECT DI.drugA, COUNT(*) APARICOES FROM GrupoMais30 GM, DrugInteraction DI GROUP BY DI.drugA;

SELECT GM.drugA, GM.drugB, CAST(GM.WEIGHT AS FLOAT)/GMA.APARICOES Confianca FROM GrupoMais30 GM, GrupoMais30_Ap GMA;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [30]:
DROP VIEW IF EXISTS GrupoMais30;
CREATE VIEW GrupoMais30 AS
SELECT * FROM DrugInteraction DI WHERE DI.weight > 30;

--Nao deu tempo de pensar em um jeito para fazer isso iterativamente para todos os medicamentos, porém neste exemplo podemos fazer na mão
--um dos casos
SELECT COUNT(*) NUMSOZINHA FROM DrugUse DU, SideEffect SE
WHERE DU.codeDrug = 96 AND DU.idPerson = SE.idPerson;

SELECT COUNT(*) FROM DrugUse DU1, DrugUse DU2, SideEffect SE 
WHERE DU1.idPerson = DU2.idPerson AND DU1.codeDrug = 96 AND DU2.codeDrug = 216 AND DU1.idPerson = SE.idPerson ;

--Assim, vemos que o medicamento A tem aprox. 4,7 vezes mais pessoas com mais SideEffects do que se comparado com aqueles que tomam o 
--medicamento A e B juntos
